In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Amplitude Analysis with simple Python
TR-999
^^^
+++
✅&nbsp;[ComPWA/RUB-EP1-AG#93](https://github.com/ComPWA/RUB-EP1-AG/issues/93)
:::
::::

# Amplitude Analysis 101

In [ ]:
%pip install -q gdown==4.7.1 matplotlib==3.7.3 numpy==1.24.4 particle==0.23.0 pylorentz==0.3.3 scipy==1.10.1

In [ ]:
%pip install --upgrade -q jupyterlab-git

In [ ]:
from __future__ import annotations

import warnings

import gdown
import numpy as np

warnings.filterwarnings("ignore")

In [ ]:
%config InlineBackend.figure_formats = ['png']

## From Data

In [ ]:
filename = gdown.cached_download(
    url="https://indico.ific.uv.es/event/6803/contributions/21223/attachments/11221/15563/Three-particles-3.dat",
    path="data/Three-particles-3.dat",
    md5="75fedf381f9b62d3210ff200fc63165f",
    quiet=True,
    verify=False,
)
data = np.loadtxt(filename)
data.shape

A python package `pylorentz` is imported to perform calculation for 4-vectors.

[pylorentz API reference](https://pylorentz.readthedocs.io/en/latest/api_reference.html)

In [ ]:
n_final_state = 3
pa_data_get, p1_data_get, p2_data_get, p3_data_get = (
    data[i::4].T for i in range(n_final_state + 1)
)
p0_data_get = p1_data_get + p2_data_get + p3_data_get
pb_data_get = p0_data_get - pa_data_get

In [ ]:
import matplotlib.pyplot as plt
from pylorentz import Momentum4

In [ ]:
pa_data = Momentum4(*pa_data_get)
pb_data = Momentum4(*pb_data_get)
p1_data = Momentum4(*p1_data_get)
p2_data = Momentum4(*p2_data_get)
p3_data = Momentum4(*p3_data_get)

system12_data = p1_data + p2_data
system23_data = p2_data + p3_data
system13_data = p1_data + p3_data

s12_data = system12_data.m2
s31_data = system13_data.m2
s23_data = system23_data.m2

In [ ]:
np.testing.assert_allclose(
    system12_data.e**2 - system12_data.p2, s12_data, rtol=1e-5
)

p12_data_get = p1_data_get + p2_data_get
np.testing.assert_allclose(
    p12_data_get[0] ** 2
    - (np.sqrt(p12_data_get[1] ** 2 + p12_data_get[2] ** 2 + p12_data_get[3] ** 2))
    ** 2,
    s12_data,
    rtol=1e-5,
)

print("test of s_12 past with relative tolerance 1e-5")

The values of $R_{12}$, $R_{23}$, and $R_{31}$ are the position of mass square that estimated form the Dalitz plot below.

In [ ]:
R12 = 1.74
R23 = 1.53
R31 = 2.45

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(10, 4), ncols=2)
fig.suptitle("Dalitz plot – scatter plot")
ax1.scatter(s12_data, s23_data, c="black", s=1e-3)
ax1.set_xlabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
ax1.set_ylabel(R"$s_{23}\;\left[\mathrm{GeV}^2\right]$")
ax1.axvline(R12, c="C0", ls="dashed", label="$R_{12}$")
ax1.axhline(R23, c="C1", ls="dashed", label="$R_{23}$")
ax1.legend()
ax2.scatter(s31_data, s12_data, c="black", s=1e-3)
ax2.set_xlabel(R"$s_{31}\;\left[\mathrm{GeV}^2\right]$")
ax2.set_ylabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
ax2.axvline(R31, c="C2", ls="dashed", label="$R_{31}$")
ax2.axhline(R12, c="C0", ls="dashed", label="$R_{12}$")
ax2.legend()
fig.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 4), ncols=3)
fig.suptitle("1D histogram of $s_{12}, s_{23}$, and $s_{31}$")
ax1.hist(s12_data, bins=100, color="black", histtype="step")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel("counts")
ax1.axvline(R12, c="C0", ls="dashed", label="$R_{12}$")
ax1.legend()

ax2.hist(s23_data, bins=100, color="black", histtype="step")
ax2.set_xlabel(R"$s_{23}$")
ax2.set_ylabel("counts")
ax2.axvline(R23, c="C1", ls="dashed", label="$R_{23}$")
ax2.legend()

ax3.hist(s31_data, bins=100, color="black", histtype="step")
ax3.set_xlabel(R"$s_{31}$")
ax3.set_ylabel("counts")
ax3.axvline(R31, c="C2", ls="dashed", label="$R_{31}$")
ax3.legend()

fig.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 4), ncols=3)
fig.suptitle("1D histogram of $s_{12}, s_{23}$, and $s_{31}$")
ax1.hist(s12_data, bins=100, color="black", histtype="step")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel("counts")
ax1.axvline(R12 - 0.25, c="C0", ls="dashed", label="$-$")
ax1.axvline(R12 + 0.25, c="C0", ls="dashed", label="$+$")
ax1.legend()

ax2.hist(s23_data, bins=100, color="black", histtype="step")
ax2.set_xlabel(R"$s_{23}$")
ax2.set_ylabel("counts")
ax2.axvline(R23 - 0.3, c="C1", ls="dashed", label="$-$")
ax2.axvline(R23 + 0.3, c="C1", ls="dashed", label="$+$")
ax2.legend()

ax3.hist(s31_data, bins=100, color="black", histtype="step")
ax3.set_xlabel(R"$s_{31}$")
ax3.set_ylabel("counts")
ax3.axvline(R31 - 0.25, c="C2", ls="dashed", label="$-$")
ax3.axvline(R31 + 0.45, c="C2", ls="dashed", label="$+$")
ax3.legend()

fig.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(R"$\theta$ and $\phi$ distribution of $p_1$")
ax1.hist(np.degrees(p1_data.theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(p1_data.phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(
    R"$\theta$ and $\phi$ distribution of $p_1$ after boost to $p_1+p_2$ system"
)
ax1.hist(np.degrees(p1_data.boost_particle(p1_data + p2_data).theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(p1_data.boost_particle(p1_data + p2_data).phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(R"$\theta$ and $\phi$ distribution of $p_2$")
ax1.hist(np.degrees(p2_data.theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(p2_data.phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(
    R"$\theta$ and $\phi$ distribution of $p_2$ after boost to $p_2+p_3$ system"
)
ax1.hist(np.degrees(p2_data.boost_particle(p2_data + p3_data).theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(p2_data.boost_particle(p2_data + p3_data).phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(R"$\theta$ and $\phi$ distribution of $p_3$")
ax1.hist(np.degrees(p3_data.theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(p3_data.phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(
    R"$\theta$ and $\phi$ distribution of $p_3$ after boost to $p_3+p_1$ system"
)
ax1.hist(np.degrees(p3_data.boost_particle(p1_data + p3_data).theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(p3_data.boost_particle(p1_data + p3_data).phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Dalitz Plot")
ax.hist2d(s12_data, s23_data, bins=100, cmin=1e-6)
ax.set_xlabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$s_{23}\;\left[\mathrm{GeV}^2\right]$")
fig.tight_layout()
plt.show()

## From model: flat phase space

In [ ]:
filename = gdown.cached_download(
    url="https://indico.ific.uv.es/event/6803/contributions/21220/attachments/11209/15505/Three-particles-flat.dat",
    path="Three-particles-flat.dat",
    md5="7624074870c22b57581e5c54a1b93754",
    quiet=True,
    verify=False,
)
data_flat = np.loadtxt(filename)
data_flat.shape

In [ ]:
n_final_state = 3
pa, p1, p2, p3 = (data_flat[i::4].T for i in range(n_final_state + 1))
p0 = p1 + p2 + p3
pb = p0 - pa

pa = Momentum4(*pa)
pb = Momentum4(*pb)
p1 = Momentum4(*p1)
p2 = Momentum4(*p2)
p3 = Momentum4(*p3)

system12 = p1 + p2
system23 = p2 + p3
system13 = p1 + p3
s12 = invariantMassSquared12 = system12.m2
s31 = invariantMassSquared13 = system13.m2
s23 = invariantMassSquared23 = system23.m2

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Dalitz plot")
ax.hist2d(s12, s23, bins=100, cmin=1)
ax.set_xlabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$s_{23}\;\left[\mathrm{GeV}^2\right]$")
fig.tight_layout()
plt.show()

```{image} https://github.com/ComPWA/strong2020-salamanca/assets/17490173/29b2514c-5683-4593-8ee7-c9a7ae64223f
:width: 33%
```

```{image} https://github.com/ComPWA/strong2020-salamanca/assets/17490173/ba309ebb-e432-4fad-802d-b6a5f43a71cd
:width: 33%
```

```{image} https://github.com/ComPWA/strong2020-salamanca/assets/17490173/867578c7-f063-449c-9ecb-a40c46e14213
:width: 33%
```

$$ 
I = |A|^2
$$

$$
A = A^{12} + A^{23} + A^{31}
$$

$$
1 \equiv \eta ; \quad  2 \equiv \pi^0 ; \quad 3 \equiv p
$$

$$
A^{12} = \frac{\sum a_m Y_2^m (\Omega_1)}{s-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} \times s^{0.5+0.9u_3}
$$

$$
A^{23} = \frac{\sum b_m Y_1^m (\Omega_2)}{s-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}} \times s^{0.5+0.9t_1}
$$

$$
A^{31} = \frac{c_0}{s-m^2_{N^*}+im_{N^*} \Gamma_{N^*}} \times s^{1.08+0.2t_2}
$$

with $s_{ij}=(p_i+p_j)^2$, $t_i=(p_a-p_i)^2$, and $u_i=(p_b-p_i)^2$

In [ ]:
def BW(s, m, Gamma):
    return 1 / (s - m**2 + complex(0, m * Gamma))

In [ ]:
total = pa + pb
total[1:].max()

In [ ]:
u3 = (pb - p3).m2
t1 = (pa - p1).m2
t2 = (pa - p2).m2

In [ ]:
print((pb - p3).e ** 2)
print((pb - p3).p ** 2)
print(u3)
print(t1)
print(t2)

In [ ]:
def p3_length(four_momentum: np.ndarray) -> np.ndarray:
    return np.sqrt(
        four_momentum[1] ** 2 + four_momentum[2] ** 2 + four_momentum[3] ** 2
    )

In [ ]:
p12 = p1 + p2
p23 = p2 + p3

In [ ]:
pa_length = p3_length(pa)
p1_length = p3_length(p1)
p2_length = p3_length(p2)
p3_length = p3_length(p3)

The Polar Angles:

$$
\theta = \arccos \frac{p_z}{|p|}
$$

In [ ]:
theta_a = np.arccos(pa[3] / pa_length)
theta_1 = np.arccos(p1[3] / p1_length)
theta_2 = np.arccos(p2[3] / p2_length)
theta_3 = np.arccos(p3[3] / p3_length)

The Azimuthal Angles:

$$
\phi = \arctan2(p_y , p_x)
$$

In [ ]:
phi_a = np.arctan2(pa[2], pa[1])
phi_1 = np.arctan2(p1[2], p1[1])
phi_2 = np.arctan2(p2[2], p2[1])
phi_3 = np.arctan2(p3[2], p3[1])

In [ ]:
import scipy as sp

$Y_l^m(\phi, \theta)$ in scipy.special.sph_harm$(m, l, \phi, \theta)$

$Y_l^m(\phi, \theta) = \sqrt{\frac{2n+1}{4\pi}\frac{(n-m)!}{(n+m)!}}e^{im\phi}P_l^m(\cos(\theta))$

here the notation of $\theta$ and $\phi$ are not using the same as in `scipy`

where 
$\phi$ is the azimuthal  from -$\pi$ to $\pi$ (in `scipy` it is $\theta$ and from 0 to $2\pi$)

$\theta$ is the polar angle from 0 to $\pi$ (in `scipy` it is $\phi$)

In [ ]:
print(sp.special.sph_harm(1, 1, -np.pi, np.pi / 2))
print(sp.special.sph_harm(1, 1, 0, np.pi / 2))

print(sp.special.sph_harm(1, 1, np.pi / 2, np.pi / 2))
print(sp.special.sph_harm(1, 1, 3 * np.pi / 2, np.pi / 2))

In [ ]:
phi_1_adj = phi_1
print(phi_1_adj)
ori = p1.boost_particle(p1 + p2).phi
adj = p1.boost_particle(p1 + p2).phi
len(phi_1)
if phi_1_adj[1] < 0:
    print("yes")
for i in range(len(adj)):
    if adj[i] < 0:
        adj[i] = adj[i] + 2 * np.pi

print(ori[4])
print(adj[4])

In [ ]:
sp.special.sph_harm(1, 1, theta_1, phi_1)
sp.special.sph_harm(0, 1, theta_1, phi_1)
sp.special.sph_harm(-1, 1, theta_1, phi_1)

In [ ]:
sp.special.sph_harm(2, 2, theta_1, phi_1)
sp.special.sph_harm(1, 2, theta_1, phi_1)
sp.special.sph_harm(0, 2, theta_1, phi_1)
sp.special.sph_harm(-1, 2, theta_1, phi_1)
sp.special.sph_harm(-2, 2, theta_1, phi_1)

In [ ]:
sp.special.sph_harm(0, 2, phi_1, theta_1)

In [ ]:
sp.special.sph_harm(0, 0, theta_1, phi_1)

Spherical harmonics 

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(R"$\theta$ and $\phi$ distribution of $system_{12}$")
ax1.hist(np.degrees(system12.theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(system12.phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(
    R"$\theta$ and $\phi$ distribution of $system_{12}$ after boost to $p_1+p_2$"
    r" system"
)
ax1.hist(np.degrees(system12.boost_particle(p1_data + p2_data).theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(system12.boost_particle(p1_data + p2_data).phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(R"$\theta$ and $\phi$ distribution of $system_{23}$")
ax1.hist(np.degrees(system23.theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(system23.phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(
    R"$\theta$ and $\phi$ distribution of $system_{23}$ after boost to $p_2+p_3$"
    r" system"
)
ax1.hist(np.degrees(system23.boost_particle(p2_data + p3_data).theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(system23.boost_particle(p2_data + p3_data).phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(R"$\theta$ and $\phi$ distribution of $system_{31}$")
ax1.hist(np.degrees(system13.theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(system13.phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)
fig.suptitle(
    R"$\theta$ and $\phi$ distribution of $system_{31}$ after boost to $p_3+p_1$"
    r" system"
)
ax1.hist(np.degrees(system13.boost_particle(p3_data + p1_data).theta), bins=100)
ax1.set_xlabel(R"$\theta$ [deg]")
ax1.set_ylabel("counts")

ax2.hist(np.degrees(system13.boost_particle(p3_data + p1_data).phi), bins=100)
ax2.set_xlabel(R"$\phi$ [deg]")
ax2.set_ylabel("counts")

plt.show()

In [ ]:
def compute_spherical_harmonics12(theta: np.ndarray, phi: np.ndarray) -> np.ndarray:
    return (
        2.5 * sp.special.sph_harm(2, 2, theta, phi)
        + 4 * sp.special.sph_harm(1, 2, theta, phi)
        + 3.5 * sp.special.sph_harm(0, 2, theta, phi)
        + 0.5 * sp.special.sph_harm(-1, 2, theta, phi)
        + 0 * sp.special.sph_harm(-2, 2, theta, phi)
    )

In [ ]:
PHI, THETA = np.meshgrid(
    np.linspace(-np.pi, +np.pi, num=1_000),
    np.linspace(0, np.pi, num=1_000),
)
Z = compute_spherical_harmonics12(PHI, THETA)

fig, axes = plt.subplots(figsize=(10, 4), ncols=2, sharey=True, dpi=120)
cmap_real = axes[0].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z.real, cmap=plt.cm.coolwarm
)
cmap_imag = axes[1].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z.imag, cmap=plt.cm.coolwarm
)

axes[0].set_xlabel(R"$\phi$ [deg]")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[0].set_title(R"Real Part of $\sum a_m Y_2^m (\Omega_1)$")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[1].set_xlabel(R"$\phi$ [deg]")
axes[1].set_title(R"Imaginary Part of $\sum a_m Y_2^m (\Omega_1)$")

cbar_real = fig.colorbar(cmap_real, ax=axes[0])
cbar_imag = fig.colorbar(cmap_imag, ax=axes[1])

fig.subplots_adjust(wspace=0.4, hspace=0.4)
fig.tight_layout()
plt.rcParams.update({"font.size": 10})
plt.show()

In [ ]:
plt.hist2d(system12.phi, system12.theta, bins=100, cmap=plt.cm.coolwarm)
plt.title("$system_{12}$")
plt.xlabel(R"$\phi$")
plt.ylabel(R"$\theta$")
plt.show()

In [ ]:
plt.hist2d(
    system12.phi,
    system12.theta,
    bins=100,
    weights=compute_spherical_harmonics12(system12.phi, system12.theta).real,
    cmap=plt.cm.coolwarm,
)
plt.title(
    "$system_{12}$ with real part of spherical harmonics as weights in histogram"
)
plt.xlabel(R"$\phi$")
plt.ylabel(R"$\theta$")
plt.show()

In [ ]:
def compute_spherical_harmonics23(theta: np.ndarray, phi: np.ndarray) -> np.ndarray:
    return (
        0.5 * sp.special.sph_harm(1, 1, theta, phi)
        + 4 * sp.special.sph_harm(0, 1, theta, phi)
        - 1.5 * sp.special.sph_harm(-1, 1, theta, phi)
    )

In [ ]:
PHI, THETA = np.meshgrid(
    np.linspace(-np.pi, +np.pi, num=1_000),
    np.linspace(0, np.pi, num=1_000),
)
Z = compute_spherical_harmonics23(PHI, THETA)

fig, axes = plt.subplots(figsize=(10, 4), ncols=2, sharey=True, dpi=120)
cmap_real = axes[0].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z.real, cmap=plt.cm.coolwarm
)
cmap_imag = axes[1].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z.imag, cmap=plt.cm.coolwarm
)

axes[0].set_xlabel(R"$\phi$ [deg]")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[0].set_title(R"Real Part of $\sum b_m Y_1^m (\Omega_2)$")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[1].set_xlabel(R"$\phi$ [deg]")
axes[1].set_title(R"Imaginary Part of $\sum b_m Y_1^m (\Omega_2)$")

cbar_real = fig.colorbar(cmap_real, ax=axes[0])
cbar_imag = fig.colorbar(cmap_imag, ax=axes[1])

fig.subplots_adjust(wspace=0.4, hspace=0.4)
fig.tight_layout()
plt.rcParams.update({"font.size": 10})
plt.show()

In [ ]:
c_0 = 0.25

In [ ]:
M12 = np.sqrt(R12)
M23 = np.sqrt(R23)
M31 = np.sqrt(R31)

In [ ]:
def intensity_dynamics(s12, s23, s31, *, M12, Gamma12, M23, Gamma23, M31, Gamma31):
    A12 = BW(s12, M12, Gamma12)
    A23 = BW(s23, M23, Gamma23)
    A31 = BW(s31, M31, Gamma31)
    return np.abs(A12 + A23 + A31) ** 2

In [ ]:
def BW_SH_dynamics(s12, s23, s31, *, M12, Gamma12, M23, Gamma23, M31, Gamma31):
    A12 = BW(s12, M12, Gamma12) * compute_spherical_harmonics12(
        p1.boost_particle(p1 + p2).phi, p1.boost_particle(p1 + p2).theta
    )
    A23 = BW(s23, M23, Gamma23) * compute_spherical_harmonics23(
        p2.boost_particle(p1 + p2).phi, p2.boost_particle(p1 + p2).theta
    )
    A31 = BW(s31, M31, Gamma31) * 1
    return np.abs(A12 + A23 + A31) ** 2

In [ ]:
def full_dynamics(s12, s23, s31, *, M12, Gamma12, M23, Gamma23, M31, Gamma31):
    A12 = (
        BW(s12, M12, Gamma12)
        * compute_spherical_harmonics12(
            p1.boost_particle(p1 + p2).phi, p1.boost_particle(p1 + p2).theta
        )
        * s12 ** (0.5 + 0.9 * u3)
    )
    A23 = (
        BW(s23, M23, Gamma23)
        * compute_spherical_harmonics23(
            p2.boost_particle(p1 + p2).phi, p2.boost_particle(p1 + p2).theta
        )
        * s23 ** (0.5 + 0.9 * t1)
    )
    A31 = BW(s31, M31, Gamma31) * c_0 * s31 ** (1.08 + 0.2 * t2)
    return np.abs(A12 + A23 + A31) ** 2

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 4), ncols=3, sharey=True)
fig.suptitle(
    "For from model: Dalitz Plot of only Breit-Wigner in the formula (middle plot)"
)
hist2 = ax2.hist2d(
    s12,
    s23,
    bins=100,
    weights=intensity_dynamics(
        s12,
        s23,
        s31,
        M12=M12,
        Gamma12=0.1,
        M23=M23,
        Gamma23=0.1,
        M31=M31,
        Gamma31=0.1,
    ),
    cmin=1e-6,
)
ax2.set_xlabel(R"$s_{12}$")
ax2.set_title("From model")

hist3 = ax3.hist2d(s12_data, s23_data, bins=100, cmin=1e-6)
ax3.set_title("From data")
ax3.set_xlabel(R"$s_{12}$")

hist1 = ax1.hist2d(s12, s23, bins=100, cmin=1e-6)
ax1.set_title("From flat")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
cbar3 = fig.colorbar(hist3[3], ax=ax3)

fig.tight_layout()
fig.show()

In [ ]:
type(
    compute_spherical_harmonics12(
        p1.boost_particle(p1 + p2).phi,
        p1.boost_particle(p1 + p2).theta,
    )
)
type(c_0)
print(c_0)
print(
    compute_spherical_harmonics12(
        p1.boost_particle(p1 + p2).phi,
        p1.boost_particle(p1 + p2).theta,
    )
)

(
    compute_spherical_harmonics12(
        p1.boost_particle(p1 + p2).phi,
        p1.boost_particle(p1 + p2).theta,
    )
    + c_0
)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 4), ncols=3, sharey=True)
fig.suptitle(
    "For from model: Dalitz Plots of only spherical harmonics (middle plot)"
)
hist2 = ax2.hist2d(
    s12,
    s23,
    bins=100,
    weights=np.abs(
        compute_spherical_harmonics12(
            p1.boost_particle(p1 + p2).phi,
            p1.boost_particle(p1 + p2).theta,
        )
        + compute_spherical_harmonics23(
            p2.boost_particle(p1 + p2).phi,
            p2.boost_particle(p1 + p2).theta,
        )
        + c_0
    )
    ** 2,
    cmin=1e-6,
)
ax2.set_xlabel(R"$s_{12}$")
ax2.set_title("From model")
hist3 = ax3.hist2d(s12_data, s23_data, bins=100, cmin=1e-6)
ax3.set_title("From data")
ax3.set_xlabel(R"$s_{12}$")
hist1 = ax1.hist2d(s12, s23, bins=100, cmin=1e-6)
ax1.set_title("From flat")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")
cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
cbar3 = fig.colorbar(hist3[3], ax=ax3)
fig.tight_layout()
fig.show()

In [ ]:
phi1_adj = p1.boost_particle(p1 + p2).phi
phi2_adj = p2.boost_particle(p1 + p2).phi
for i in range(len(phi1_adj)):
    if phi1_adj[i] < 0:
        phi1_adj[i] = phi1_adj[i] + 2 * np.pi
    if phi2_adj[i] < 0:
        phi2_adj[i] = phi2_adj[i] + 2 * np.pi


fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)
fig.suptitle("Dalitz Plots of only spherical harmonics")
hist1 = ax1.hist2d(
    s12,
    s23,
    bins=100,
    weights=np.abs(
        compute_spherical_harmonics12(
            phi1_adj,
            p1.boost_particle(p1 + p2).theta,
        )
        + compute_spherical_harmonics23(
            phi2_adj,
            p2.boost_particle(p1 + p2).theta,
        )
        + c_0
    )
    ** 2,
    cmin=1e-6,
)
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")
ax1.set_title("From model")
hist2 = ax2.hist2d(s12_data, s23_data, bins=100, cmin=1e-6)
ax2.set_title("From data")
ax2.set_xlabel(R"$s_{12}$")
ax2.set_ylabel(R"$s_{23}$")
cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
fig.tight_layout()
fig.show()

In [ ]:
print(
    np.abs(
        compute_spherical_harmonics12(
            p1.boost_particle(p1 + p2).phi,
            p1.boost_particle(p1 + p2).theta,
        )
        + compute_spherical_harmonics23(
            p2.boost_particle(p1 + p2).phi,
            p2.boost_particle(p1 + p2).theta,
        )
        + c_0
    )
    ** 2
)

print(
    np.abs(
        compute_spherical_harmonics12(
            phi1_adj,
            p1.boost_particle(p1 + p2).theta,
        )
        + compute_spherical_harmonics23(
            phi2_adj,
            p2.boost_particle(p1 + p2).theta,
        )
        + c_0
    )
    ** 2
)

In [ ]:
print(phi1_adj)
print(p1.boost_particle(p1 + p2).phi)

print(
    compute_spherical_harmonics12(
        4.54405994,
        (p1.boost_particle(p1 + p2).theta)[1],
    )
)

compute_spherical_harmonics12(
    -1.73912537,
    (p1.boost_particle(p1 + p2).theta)[1],
)

In [ ]:
intensity_dynamics(
    s12,
    s23,
    s31,
    M12=M12,
    Gamma12=0.1,
    M23=M23,
    Gamma23=0.1,
    M31=M31,
    Gamma31=0.1,
)

In [ ]:
full_dynamics(
    s12,
    s23,
    s31,
    M12=M12,
    Gamma12=0.1,
    M23=M23,
    Gamma23=0.1,
    M31=M31,
    Gamma31=0.1,
)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 4), ncols=3, sharey=True)
fig.suptitle(
    R"For from model: Dalitz Plots of Breit-Wigner $\times$ Spherical Harmonics"
    r" (middle plot)"
)
hist2 = ax2.hist2d(
    s12,
    s23,
    bins=100,
    weights=BW_SH_dynamics(
        s12,
        s23,
        s31,
        M12=M12,
        Gamma12=0.1,
        M23=M23,
        Gamma23=0.1,
        M31=M31,
        Gamma31=0.1,
    ),
    cmin=1e-6,
)
ax2.set_xlabel(R"$s_{12}$")
ax2.set_title("From model")


hist3 = ax3.hist2d(s12_data, s23_data, bins=100, cmin=1e-6)
ax3.set_title("From data")
ax3.set_xlabel(R"$s_{12}$")

hist1 = ax1.hist2d(s12, s23, bins=100, cmin=1e-6)
ax1.set_title("From flat")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
cbar3 = fig.colorbar(hist3[3], ax=ax3)

fig.tight_layout()
fig.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 4), ncols=3, sharey=True)
fig.suptitle(
    "For from model: : Dalitz Plots of full expression of the formula (middle plot)"
)
hist2 = ax2.hist2d(
    s12,
    s23,
    bins=100,
    weights=full_dynamics(
        s12,
        s23,
        s31,
        M12=M12,
        Gamma12=0.1,
        M23=M23,
        Gamma23=0.1,
        M31=M31,
        Gamma31=0.1,
    ),
    cmin=1e-8,
)
ax2.set_xlabel(R"$s_{12}$")
ax2.set_title("From model")
hist3 = ax3.hist2d(s12_data, s23_data, bins=100, cmin=1e-6)
ax3.set_title("From data")
ax3.set_xlabel(R"$s_{12}$")
hist1 = ax1.hist2d(s12, s23, bins=100, cmin=1e-6)
ax1.set_title("From flat")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")
cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
cbar3 = fig.colorbar(hist3[3], ax=ax3)
fig.tight_layout()
fig.show()

In [ ]:
full_dynamics(
    s12,
    s23,
    s31,
    M12=M12,
    Gamma12=0.1,
    M23=M23,
    Gamma23=0.1,
    M31=M31,
    Gamma31=0.1,
)

In [ ]:
BW(s12, M12, 0.1) * compute_spherical_harmonics12(
    p1.boost_particle(p1 + p2).phi, p1.boost_particle(p1 + p2).theta
)

In [ ]:
print(s12_data)
print(s12)

In [ ]:
s12 ** (0.5 + 0.9 * u3)

In [ ]:
0.5 + 0.9 * u3

In [ ]:
u3

In [ ]:
s12 ** (0.5 + 0.9 * u3)

In [ ]:
BW(s12, M12, 0.1)

In [ ]:
compute_spherical_harmonics12(
    p1.boost_particle(p1 + p2).phi, p1.boost_particle(p1 + p2).theta
)

In [ ]:
BW(s12, M12, 0.1) * compute_spherical_harmonics12(
    p1.boost_particle(p1 + p2).phi, p1.boost_particle(p1 + p2).theta
)